In [ ]:
from dataDownloader  import DataDownloader
from db_financialStatement import DB_FinancialStatement
from db_nyse import DB_NYSE
from db_stock import DB_Stock
from portfolio import Portfolio
from assetAllocation import AssetAllocation
from matplotlib.dates import relativedelta
from datetime import datetime, timedelta
from commonHelper import EDateType


import datetime
import dataDownloader
import yfinance as yf;
import pandas as pd
import commonHelper
import numpy as np
import os
from scipy.stats.mstats import winsorize


start_date = '2014-12-01'
end_date = '2025-09-02'


In [ ]:

symbols = ['AAPL','AMD','A','TSLA',]
df = DB_FinancialStatement.get_fs_data_static(symbols)

grouped = df.groupby('Quarter')
for q, group in grouped:
    
    display(group['PBR'].median())

,Date,Quarter,Symbol,Sector,MarketCap,Close,NetIncome,OperatingIncome,PER,PBR,PCR,PSR,PFCR,LiquidationValue,EV/EBIT,OperatingCashFlow,CommonStockIssuance,NetCommonStockIssuance,GP/A,ROA,CurrentRatio,DebtToEquityRatio,AssetTurnoverRatio,GrossProfitMargin,TotalAssetsGrowth,OperatingIncomeGrowth,NetIncomeGrowth,TotalDebtGrowth
5,2024-03-31,2024-Q1,AAPL,Technology,2.651887e+12,171.48,2.363600e+10,2.790000e+10,112.196935,35.742604,116.874692,29.220927,107.424725,-134801000000.0,96.39164,2.269000e+10,None,-2.320500e+10,0.12528,0.070051,1.037102,0.629965,0.268969,0.465781,None,None,NaN,None
11,2024-03-31,2024-Q1,AMD,Technology,2.958231e+11,180.49,1.230000e+08,3.600000e+07,2405.065935,5.263944,567.798676,54.051363,446.188703,5387000000.0,3289.731573,5.210000e+08,None,-1.330000e+08,0.037705,0.001812,2.638863,0.051765,0.08061,0.467751,None,None,NaN,None
17,2024-03-31,2024-Q1,TSLA,Consumer Cyclical,6.124524e+11,175.79,1.390000e+09,1.171000e+09,440.613209,9.51338,2530.794876,28.752282,202.865969,6489000000.0,303.207923,2.420000e+08,None,NaN,0.033838,0.012726,1.715784,0.147668,0.195018,0.173513,None,None,NaN,None


np.float64(9.513379725993351)

,Date,Quarter,Symbol,Sector,MarketCap,Close,NetIncome,OperatingIncome,PER,PBR,PCR,PSR,PFCR,LiquidationValue,EV/EBIT,OperatingCashFlow,CommonStockIssuance,NetCommonStockIssuance,GP/A,ROA,CurrentRatio,DebtToEquityRatio,AssetTurnoverRatio,GrossProfitMargin,TotalAssetsGrowth,OperatingIncomeGrowth,NetIncomeGrowth,TotalDebtGrowth
0,2024-04-30,2024-Q2,A,Healthcare,4.015272e+10,137.04,3.080000e+08,3.630000e+08,130.365974,6.461654,120.578739,25.526205,92.093394,-439000000.0,104.15665,3.330000e+08,None,-2.300000e+08,0.07885,0.028371,2.146578,0.306108,0.144897,0.544183,None,None,NaN,None
6,2024-06-30,2024-Q2,AAPL,Technology,3.232638e+12,210.62,2.144800e+10,2.535200e+10,150.719782,48.459523,112.018778,37.686535,104.248376,-139469000000.0,129.068353,2.885800e+10,None,-2.652200e+10,0.119652,0.064678,0.95298,0.662533,0.258667,0.462572,-0.017187,-0.091326,-0.092571,-0.031418
12,2024-06-30,2024-Q2,AMD,Technology,2.655378e+11,162.21,2.650000e+08,2.690000e+08,1002.029321,4.696625,447.787133,45.507758,355.472249,6117000000.0,810.008549,5.930000e+08,None,-4.490000e+08,0.042188,0.003904,2.819209,0.038536,0.085953,0.490831,-0.000133,6.472222,1.154472,-0.251167
18,2024-06-30,2024-Q2,TSLA,Consumer Cyclical,6.888203e+11,197.88,1.400000e+09,2.227000e+09,492.014486,10.363186,190.703289,27.01256,117.066669,7408000000.0,358.043396,3.612000e+09,None,NaN,0.040574,0.012408,1.910527,0.174289,0.226,0.179529,0.033014,0.901793,0.007194,0.262738


np.float64(8.41242041234601)

,Date,Quarter,Symbol,Sector,MarketCap,Close,NetIncome,OperatingIncome,PER,PBR,PCR,PSR,PFCR,LiquidationValue,EV/EBIT,OperatingCashFlow,CommonStockIssuance,NetCommonStockIssuance,GP/A,ROA,CurrentRatio,DebtToEquityRatio,AssetTurnoverRatio,GrossProfitMargin,TotalAssetsGrowth,OperatingIncomeGrowth,NetIncomeGrowth,TotalDebtGrowth
1,2024-07-31,2024-Q3,A,Healthcare,4.114740e+10,141.40,2.820000e+08,3.330000e+08,145.912766,6.970591,91.034071,26.075665,75.638603,-837000000.0,115.891507,4.520000e+08,None,-5.850000e+08,0.077756,0.025646,1.781499,0.364677,0.143507,0.541825,0.012896,-0.082645,-0.084416,0.147105
7,2024-09-30,2024-Q3,AAPL,Technology,3.551596e+12,233.00,1.473600e+10,2.959100e+10,241.014909,62.363401,132.467856,37.412785,119.505895,-155043000000.0,121.423869,2.681100e+10,None,-2.508300e+10,0.120223,0.040375,0.867313,0.747225,0.260096,0.462225,0.100624,0.167206,-0.312943,0.052565
13,2024-09-30,2024-Q3,AMD,Technology,2.684349e+11,164.08,7.710000e+08,7.240000e+08,348.164565,4.710623,427.444076,39.365725,353.203789,6084000000.0,350.169579,6.280000e+08,None,-7.100000e+08,0.049098,0.011072,2.498,0.038123,0.097923,0.501393,0.025779,1.69145,1.909434,-0.003118
19,2024-09-30,2024-Q3,TSLA,Consumer Cyclical,9.149201e+11,261.63,2.167000e+09,2.772000e+09,422.205865,13.083184,146.270201,36.332305,93.66504,7237000000.0,310.867563,6.255000e+09,None,NaN,0.041693,0.018081,1.843837,0.169973,0.210109,0.198435,0.062216,0.244724,0.547857,0.021414


...

np.float64(12.910204950715919)

,Date,Quarter,Symbol,Sector,MarketCap,Close,NetIncome,OperatingIncome,PER,PBR,PCR,PSR,PFCR,LiquidationValue,EV/EBIT,OperatingCashFlow,CommonStockIssuance,NetCommonStockIssuance,GP/A,ROA,CurrentRatio,DebtToEquityRatio,AssetTurnoverRatio,GrossProfitMargin,TotalAssetsGrowth,OperatingIncomeGrowth,NetIncomeGrowth,TotalDebtGrowth
3,2025-01-31,2025-Q1,A,Healthcare,4.348624e+10,151.52,3.180000e+08,3.760000e+08,136.749182,7.215238,100.896148,25.869268,82.360303,-1780000000.0,114.891747,4.310000e+08,None,-9.000000e+07,0.075457,0.026691,2.197432,0.358758,0.141095,0.534801,0.00574,-0.078431,-0.094017,-0.007965
9,2025-03-31,2025-Q1,AAPL,Technology,3.344411e+12,222.13,2.478000e+10,2.958900e+10,134.964144,50.069038,139.629738,35.071797,123.761666,-145763000000.0,114.708151,2.395200e+10,None,-2.589800e+10,0.135454,0.074811,0.82087,0.675459,0.287891,0.470506,-0.037351,-0.309185,-0.317919,0.014329
15,2025-03-31,2025-Q1,AMD,Technology,1.670552e+11,102.74,7.090000e+08,8.060000e+08,235.620931,2.886184,177.907604,22.459699,145.139218,7926000000.0,194.646438,9.390000e+08,None,-7.790000e+08,0.052215,0.009909,2.803453,0.077433,0.103955,0.502286,0.033571,-0.237465,0.470954,1.138788
21,2025-03-31,2025-Q1,TSLA,Consumer Cyclical,9.125024e+11,259.16,4.090000e+08,4.930000e+08,2231.057115,12.223251,423.238571,47.194329,250.137708,9696000000.0,1306.815235,2.156000e+09,None,NaN,0.025202,0.003269,1.996068,0.16465,0.154543,0.163072,0.024912,-0.689937,-0.826401,-0.036336


np.float64(9.719244568361685)

,Date,Quarter,Symbol,Sector,MarketCap,Close,NetIncome,OperatingIncome,PER,PBR,PCR,PSR,PFCR,LiquidationValue,EV/EBIT,OperatingCashFlow,CommonStockIssuance,NetCommonStockIssuance,GP/A,ROA,CurrentRatio,DebtToEquityRatio,AssetTurnoverRatio,GrossProfitMargin,TotalAssetsGrowth,OperatingIncomeGrowth,NetIncomeGrowth,TotalDebtGrowth
4,2025-04-30,2025-Q2,A,Healthcare,3.066600e+10,107.60,2.150000e+08,3.000000e+08,142.632558,4.997718,138.760181,18.384892,91.540299,-1814000000.0,113.062284,2.210000e+08,None,-1.650000e+08,0.071229,0.017684,2.091451,0.368477,0.137194,0.519185,0.02048,-0.202128,-0.323899,0.039251
10,2025-06-30,2025-Q2,AAPL,Technology,3.066922e+12,205.17,2.343400e+10,2.820200e+10,130.874891,46.588519,110.0557,32.614341,97.894034,-143174000000.0,110.391043,2.786700e+10,None,-2.107500e+10,0.131881,0.070692,0.867992,0.685944,0.283672,0.464907,0.000791,-0.046876,-0.054318,0.035769
16,2025-06-30,2025-Q2,AMD,Technology,2.312970e+11,141.90,8.720000e+08,-1.340000e+08,265.248853,3.876594,115.015912,30.097202,100.870911,9364000000.0,-6369.888889,2.011000e+09,None,-5.240000e+08,0.040885,0.011655,2.491009,0.061797,0.102713,0.398048,0.045702,-1.166253,0.229901,-0.178609
22,2025-06-30,2025-Q2,TSLA,Consumer Cyclical,1.117846e+12,317.66,1.172000e+09,9.230000e+08,953.793123,14.458514,440.096669,49.690858,226.559696,10638000000.0,669.233969,2.540000e+09,None,NaN,0.030163,0.009116,2.037223,0.159571,0.174975,0.172386,0.027623,0.872211,1.865526,0.000457


np.float64(9.728116117956343)